In [93]:
from typing import Tuple
import pandas as pd 
import random 
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

In [94]:
class ProcessingData:
    # zwracam dataframe w losowej kolejności i resetuje indeksy
    @staticmethod
    def shuffleDF(df: pd.DataFrame) -> pd.DataFrame:
        return df.iloc[random.sample(range(len(df)), len(df))].reset_index(drop=True)

    # normalizuje każdą kolumnę w dataframe (według przekazanej listy) metodą min max
    @staticmethod
    def normalizeDF(df: pd.DataFrame, columnNames: list) -> pd.DataFrame:
        for columnName in columnNames:
            df[columnName] = (df[columnName]-df[columnName].min())/(df[columnName].max()-df[columnName].min())
        return df

    # zwracam wiersze gdzie index jest mniejszy od długość df*wielkość df i wiersze gdzie index jest równy lub większy od długość *wielkość df 
    @staticmethod
    def splitDF(df: pd.DataFrame, trainSize: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        return df[df.index < int(len(df)*trainSize)], df[df.index >= int(len(df)*trainSize)]

    @staticmethod
    def processData(df: pd.DataFrame, columnNames: list, trainSize: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        df = ProcessingData.shuffleDF(df)
        df = ProcessingData.normalizeDF(df, columnNames)
        return ProcessingData.splitDF(df, trainSize)

In [95]:

class KNN:
    def __init__(self, k: int):
        self.k = k
    
    #liczę dystatns przy użyciu metryki taksówkowej
    @staticmethod
    def dst(x: list, y: list) -> float:
        return sum([(abs(xi-yi))
                    for xi,yi in zip(x,y)])

    #do obiektu wpisuje dane treningowe
    def fit(self, df: pd.DataFrame):
        self.df = df
    
    #dla danego obiektu zwraca najczęściej pojawianego się sąsiada
    def predict(self, point: pd.DataFrame) -> str:
        #tworzę słownik z typami
        types = {}
        for v in range(11):
            types[v] = 0
        result = []
        # liczę odległości od każdego punktu
        for sample in self.df.values:
            result.append([KNN.dst(sample[:-1], point), sample[-1]])
        #sortuje odległości
        result.sort(key=lambda x:x[0])
        # zliczam type k najbliższych odległości
        for i in range(self.k):
            types[result[i][1]] += 1
        return max(types, key=types.get)
    
    #liczę wynik dla zbioru testowego
    def score(self, test_X: pd.DataFrame) -> float:
        good = 0
        bad = 0
        for sample in test_X.values:
            # sprawdzam czy predict zwrócił dobry typ
            if (x:=self.predict(sample)) == sample[-1]:
                print(x, sample[-1])
                good += 1
            else:
                bad +=1
        return good/(bad+good)*100

In [96]:
# znormalizowane
seeds = pd.read_csv("seeds.csv", sep=",")
X_train, X_test = ProcessingData.processData(seeds, seeds.columns[:-1], 0.8)
knn = KNN(4)
knn.fit(X_train)
print(knn.score(X_test))

32.5


In [97]:
# bez normalizacji
seeds = pd.read_csv("seeds.csv", sep=",")
seeds = ProcessingData.shuffleDF(seeds)
X_train, X_test = ProcessingData.splitDF(seeds, 0.8)
knn = KNN(4)
knn.fit(X_train)
print(knn.score(X_test))

30.0


In [98]:
digits = datasets.load_digits()

In [99]:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

In [100]:

clf = KNN(4)

In [101]:
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.2
)

In [102]:
clf.fit(pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis=1))


In [103]:
clf.score(pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis=1))

10.22964509394572

In [104]:
print(clf.predict(pd.DataFrame(X_test[10])))
print(y_test[10])

0
8


In [105]:
print(X_test[100])
print(pd.DataFrame(X_test[100]))

[ 0.  0.  8. 12. 12. 14.  3.  0.  0.  0. 11. 11. 10. 16.  2.  0.  0.  0.
  0.  0.  9. 13.  0.  0.  0.  0.  0. 14. 16. 13.  0.  0.  0.  0.  0.  8.
  8. 16.  4.  0.  0.  0.  3.  0.  0. 16.  4.  0.  0.  1. 16.  9.  9. 15.
  2.  0.  0.  1. 11. 14. 15.  3.  0.  0.]
       0
0    0.0
1    0.0
2    8.0
3   12.0
4   12.0
..   ...
59  14.0
60  15.0
61   3.0
62   0.0
63   0.0

[64 rows x 1 columns]


In [106]:
import matplotlib.pyplot as plt

In [107]:
for i in range(20,40):
    print(clf.predict(pd.DataFrame(X_test[i])), end=", ")
    print(y_test[i])

0, 5
0, 8
0, 8
0, 9
0, 2
0, 7
0, 8
0, 5
0, 0
0, 9
0, 4
0, 1
0, 2
0, 6
0, 4
0, 5
0, 6
0, 0
0, 1
0, 1
